In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
meta_train = json.load(open('data/train_annotations.json'))
meta_train.keys()

dict_keys(['info', 'images', 'annotations', 'categories'])

In [3]:
images_train = pd.DataFrame(meta_train['images'])
annotations_train = pd.DataFrame(meta_train['annotations'])

In [4]:
images_train.head()

,file_name,height,id,location,rights_holder,width
0,train_val/5968c0f9-23d2-11e8-a6a3-ec086b02610b...,1494,5968c0f9-23d2-11e8-a6a3-ec086b02610b,26,Erin Boydston,2048
1,train_val/5a0b016f-23d2-11e8-a6a3-ec086b02610b...,1494,5a0b016f-23d2-11e8-a6a3-ec086b02610b,26,Erin Boydston,2048
2,train_val/59b93afb-23d2-11e8-a6a3-ec086b02610b...,1494,59b93afb-23d2-11e8-a6a3-ec086b02610b,38,Justin Brown,2048
3,train_val/59641f56-23d2-11e8-a6a3-ec086b02610b...,1494,59641f56-23d2-11e8-a6a3-ec086b02610b,38,Justin Brown,2048
4,train_val/5a1e5306-23d2-11e8-a6a3-ec086b02610b...,1494,5a1e5306-23d2-11e8-a6a3-ec086b02610b,33,Justin Brown,2048


In [5]:
annotations_train.head()

,category_id,id,image_id
0,0,825024ef-23d2-11e8-a6a3-ec086b02610b,5968c0f9-23d2-11e8-a6a3-ec086b02610b
1,1,82937b1d-23d2-11e8-a6a3-ec086b02610b,5a0b016f-23d2-11e8-a6a3-ec086b02610b
2,1,825c44a2-23d2-11e8-a6a3-ec086b02610b,59b93afb-23d2-11e8-a6a3-ec086b02610b
3,1,821f3f55-23d2-11e8-a6a3-ec086b02610b,59641f56-23d2-11e8-a6a3-ec086b02610b
4,1,823c54b7-23d2-11e8-a6a3-ec086b02610b,5a1e5306-23d2-11e8-a6a3-ec086b02610b


In [6]:
df_train = pd.merge(images_train, annotations_train[['category_id', 'image_id']], left_on='id', right_on='image_id').drop('image_id', axis=1)

In [7]:
df_train.head()

,file_name,height,id,location,rights_holder,width,category_id
0,train_val/5968c0f9-23d2-11e8-a6a3-ec086b02610b...,1494,5968c0f9-23d2-11e8-a6a3-ec086b02610b,26,Erin Boydston,2048,0
1,train_val/5a0b016f-23d2-11e8-a6a3-ec086b02610b...,1494,5a0b016f-23d2-11e8-a6a3-ec086b02610b,26,Erin Boydston,2048,1
2,train_val/59b93afb-23d2-11e8-a6a3-ec086b02610b...,1494,59b93afb-23d2-11e8-a6a3-ec086b02610b,38,Justin Brown,2048,1
3,train_val/59641f56-23d2-11e8-a6a3-ec086b02610b...,1494,59641f56-23d2-11e8-a6a3-ec086b02610b,38,Justin Brown,2048,1
4,train_val/5a1e5306-23d2-11e8-a6a3-ec086b02610b...,1494,5a1e5306-23d2-11e8-a6a3-ec086b02610b,33,Justin Brown,2048,1


In [8]:
df_train.location.value_counts()

26    28670
57    12805
23    12375
38     9751
46     6056
43     4082
33     3928
61     2268
7      1837
0      1784
51     1489
27     1435
41     1375
34     1342
28     1279
36     1216
40     1060
64      963
6       948
18      783
58      767
10      722
14      624
24      616
44      590
56      556
13      493
53      428
42      407
60      405
      ...  
55      263
22      259
1       248
39      246
37      244
2       235
63      220
4       211
20      198
29      196
25      155
35      152
12      143
48      143
17      138
31      117
62      116
45      106
30      105
16       67
9        66
21       52
50       40
8        24
52       23
11       21
54       19
49       12
15        9
47        8
Name: location, Length: 65, dtype: int64

In [9]:
df_train.height.value_counts()

1494    101523
1536      2755
584       1447
1920       703
Name: height, dtype: int64

In [10]:
df_train.width.value_counts()

2048    104278
800       1447
2560       703
Name: width, dtype: int64

In [11]:
2048, 1494
0.75 * 600

450.0

In [12]:
df_train.category_id.value_counts()

1    55902
0    50526
Name: category_id, dtype: int64

In [13]:
def create_map_files(src, dest):
    data = json.load(open(src))
    imgs = pd.DataFrame(data['images'])
    if dest is not 'test':
        print('In if: {}'.format(dest))
        annotations = pd.DataFrame(data['annotations'])
        annotations = annotations.replace({'category_id': {0: 'no_animal', 1: 'animal'}})
        df = pd.merge(imgs, annotations[['category_id', 'image_id']], left_on='id', right_on='image_id').drop('image_id', axis=1)
        df = df[['file_name', 'location', 'category_id']]
    else:
        print('In else: {}'.format(dest))
        df = imgs[['file_name', 'id', 'location']]
    df.to_csv('data/{}.csv'.format(dest), index=False)

In [14]:
map_files = [
    ('data/train_annotations.json', 'train'),
    ('data/val_annotations.json', 'val'),
#     ('data/test_information.json', 'test')
]

for src, dest in map_files:
    create_map_files(src, dest)

In if: train
In if: val


In [15]:
train = pd.read_csv('data/train.csv')

In [16]:
train.head()

,file_name,location,category_id
0,train_val/5968c0f9-23d2-11e8-a6a3-ec086b02610b...,26,no_animal
1,train_val/5a0b016f-23d2-11e8-a6a3-ec086b02610b...,26,animal
2,train_val/59b93afb-23d2-11e8-a6a3-ec086b02610b...,38,animal
3,train_val/59641f56-23d2-11e8-a6a3-ec086b02610b...,38,animal
4,train_val/5a1e5306-23d2-11e8-a6a3-ec086b02610b...,33,animal


In [18]:
!ls data/

test_information.json	train.csv  val_annotations.json
train_annotations.json	train_val  val.csv
